In [7]:
%load_ext autoreload
%autoreload 2

import altair as alt
from embeddings_analysis import EmbeddingsLoader

from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.manifold import TSNE
from umap import UMAP


alt.data_transformers.disable_max_rows()
alt.renderers.set_embed_options(theme='dark')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


RendererRegistry.enable('default')

# OLMo-2-1124-7B

The model has been chosen as one of the targets of this analysis because of its inclination towards research and hackability. Like the other models considered, OLMo uses a BPE tokenizer in which the 0-999 range seems to be hardcoded to be encoded with a single token for each number.
Only numbers in this range are considered, even though there might be bigger integers that get encoded with a single token by the BPE tokenizer.

We check the numbers that get encoded in a single embedding vector by running the tokenizer on all the numbers in the range until we find the first one that gets encoded with more than one token.

In [4]:
model_id = "allenai/OLMo-2-1124-7B"
loader = EmbeddingsLoader(model_id)
loader.smallest_multitoken_number()

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

1000

One of the goal of this analysis is to find structures in the embeddings.

Hypotheses:
- The representation of different models converges to similar structures
- Numerical embeddings have a representation that favors numerical calculation tasks
- There are optimal structures that the embeddings converge towards in the pursuit of certain tasks
    - This is what might allow to do away with learning and instead provide the structure directly by calculating it (providing concrete structures).

It's also notable that the choice of having specific tokens for the numbers in the range 0-999 bias the model toward a direct representation of positive integers, possibly negating symmetries with negative numbers.

In [ ]:
# Loading the number embeddings and 1000 random embeddings for comparison

number_embeddings = loader.numbers()
random_embeddings = loader.random()

Dimensionality reduction techniques are employed to visualize the structures that might emerge from the embeddings. They also are compared to a visualization of random embeddings to show that the structure is specific to the number embeddings.

In [ ]:
number_pca = number_embeddings.dim_reduction(PCA(n_components=100))
random_pca = random_embeddings.dim_reduction(PCA(n_components=100))

alt.hconcat(number_pca.plot(), random_pca.plot()).resolve_scale(color="independent")

NameError: name 'PCA' is not defined

In [7]:
plots

{'explained_variance': alt.Chart(...),
 'cumulative_variance': alt.LayerChart(...),
 'projection': alt.Chart(...),
 'consecutive_distances': alt.Chart(...),
 'component_patterns': alt.Chart(...),
 'correlation_heatmap': alt.Chart(...),
 'ones_digit': alt.Chart(...),
 'tens_digit': alt.Chart(...),
 'hundreds_digit': alt.Chart(...),
 'digit_length': alt.Chart(...),
 'special_numbers': alt.LayerChart(...)}

In [8]:
alt.hconcat(plots['explained_variance'], plots['cumulative_variance'])

alt.HConcatChart(...)

In [13]:
alt.vconcat(plots['projection'], plots['ones_digit'], plots['tens_digit'], plots['hundreds_digit'], plots['digit_length'])

alt.VConcatChart(...)

In [10]:
plots['projection']

alt.Chart(...)

In [11]:
plots['component_patterns']

alt.Chart(...)